Copyright 2023-2023 Lawrence Livermore National Security, LLC and other MuyGPyS
Project Developers. See the top-level COPYRIGHT file for details.

SPDX-License-Identifier: MIT

# Shear Kernel Tutorial

This notebook demonstrates how to use the specialized lensing shear kernel (hard-coded to RBF at the moment).

⚠️ _Note that this is still an experimental feature._ ⚠️

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from matplotlib.colors import LogNorm
from MuyGPyS._src.gp.tensors import _crosswise_differences, _pairwise_differences
from MuyGPyS.gp import MuyGPS
from MuyGPyS.gp.deformation import Isotropy, l2, F2
from MuyGPyS.gp.hyperparameter import Parameter
from MuyGPyS.gp.kernels.experimental import ShearKernel
from MuyGPyS.gp.tensors import make_predict_tensors
from MuyGPyS.neighbors import NN_Wrapper
from MuyGPyS.gp.noise import HomoscedasticNoise

This is required to import the implementation from Bob Armstrong's original repository.
Must set `shear_kernel_dir = "path/to/local/shear_kernel/"` for things to run properly.

In [ ]:
import importlib.util
import sys
import torch
# introduce a variable for path/to/shear_kernel
shear_kernel_dir = "../../../projects/shear_kernel/"
spec_analytic = importlib.util.spec_from_file_location("analytic_kernel", shear_kernel_dir + "analytic_kernel.py") 
bar = importlib.util.module_from_spec(spec_analytic)
sys.modules["analytic_kernel"] = bar
spec_analytic.loader.exec_module(bar)
from analytic_kernel import shear_kernel

We will set a random seed here for consistency when building docs.
In practice we would not fix a seed.

In [ ]:
np.random.seed(0)

Here we build some simple data, which is mean to represent a grid of sky coordinates.

In [ ]:
n = 25  # number of galaxies on a side
xmin = 0
xmax = 1
ymin = 0
ymax = 1

xx = np.linspace(xmin, xmax, n)
yy = np.linspace(ymin, ymax, n)

x, y = np.meshgrid(xx, yy)
features = np.vstack((x.flatten(), y.flatten())).T
data_count = features.shape[0]
diffs = _pairwise_differences(features)
length_scale = 0.5

Use an Isotropic distance functor.

In [ ]:
dist_fn = Isotropy(
    metric=F2,
    length_scale=Parameter(length_scale),
)

Here we construct a shear value kernel (partial differential components of RBF), as well as the original RBF kernel using Bob's implementation.

In [ ]:
def original_shear(X1, X2=None, length_scale=1.0):
    if X2 is None:
        X2 = X1
    n1, _ = X1.shape
    n2, _ = X2.shape
    vals = np.zeros((3 * (n1), 3 * (n2)))
    vals[:] = np.nan
    for i, (ix, iy) in enumerate(X1):
        for j, (jx, jy) in enumerate(X2):
            tmp = shear_kernel(ix, iy, jx, jy, b=length_scale)
            for a in range(3):
                for b in range(3):
                    vals[(a * n1) + i, (b * n2) + j] = tmp[a, b]
            
    return vals

In [ ]:
Kin_analytic = original_shear(features, length_scale=length_scale)

Here we do the same using the MuyGPyS implementation. Note the increased efficiency.

In [ ]:
Kin_muygps = ShearKernel(deformation=dist_fn)(diffs)

`Kin_muygps` is a more generalized tensor, so we need to flatten it to a conforming shape.

In [ ]:
Kin_flat = Kin_muygps.reshape(data_count * 3, data_count * 3)

In [ ]:
print(f"Kin_analytic.shape = {Kin_analytic.shape}")
print(f"Kin_muygps.shape = {Kin_muygps.shape}")
print(f"Kin_flat.shape = {Kin_flat.shape}")

Do the two implementations agree?

In [ ]:
np.allclose(Kin_analytic, Kin_flat)

In [ ]:
Kin_residual = np.abs(Kin_analytic - Kin_flat)
print(f"Kin residual max: {np.max(Kin_residual)}, min: {np.min(Kin_residual)}, mean : {np.mean(Kin_residual)}")

Plot results of the baseline and internal implementations. 

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12, 4))
axes[0].set_title("original shear kernel")
axes[0].imshow(Kin_analytic)
axes[1].set_title("MuyGPyS shear kernel")
axes[1].imshow(Kin_flat)
axes[2].set_title("Residual")
im = axes[2].imshow(Kin_residual, norm=LogNorm())
fig.colorbar(im, ax=axes[2])
plt.show()

Now we perform a similar analysis for the cross-covariance.

In [ ]:
split = 200
X1 = features[:split]
X2 = features[split:]
n1, _ = X1.shape
n2, _ = X2.shape
crosswise_diffs = _crosswise_differences(X1, X2)
print(X1.shape, X2.shape, crosswise_diffs.shape)

In [ ]:
Kcross_analytic = original_shear(X1, X2, length_scale=length_scale)

In [ ]:
Kcross_muygps = ShearKernel(deformation=dist_fn)(crosswise_diffs, adjust=False)

In [ ]:
Kcross_flat = Kcross_muygps.reshape(n1 * 3, n2 * 3)

In [ ]:
print(f"Kcross_analytic.shape = {Kcross_analytic.shape}")
print(f"Kcross_muygps.shape = {Kcross_muygps.shape}")
print(f"Kcross_flat.shape = {Kcross_flat.shape}")

In [ ]:
np.allclose(Kcross_analytic, Kcross_flat)

In [ ]:
Kcross_residual = np.abs(Kcross_analytic - Kcross_flat)
print(f"Kcross residual max: {np.max(Kcross_residual)}, min: {np.min(Kcross_residual)}, mean : {np.mean(Kcross_residual)}")

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12, 4))
axes[0].set_title("original shear kernel")
axes[0].imshow(Kcross_analytic)
axes[1].set_title("MuyGPyS shear kernel")
axes[1].imshow(Kcross_flat)
axes[2].set_title("Residual")
im = axes[2].imshow(Kcross_residual, norm=LogNorm())
fig.colorbar(im, ax=axes[2])
plt.show()

Runtime comparison of the two implementations:

In [ ]:
if False:
    %timeit original_shear(features)
    %timeit ShearKernel(deformation=dist_fn)(diffs)

Now we will test the `posterior_mean` of the analytic and muygps implementations.
First, we set up an arbitrary taget array.
Targets should be square matrices like a grid of a swath of sky.
Ulitimately, the target array will have shape `(625,3)`, given `n=25` above.

In [ ]:
targets_grid_x, targets_grid_y = np.meshgrid(
        np.linspace(1, 10, n),
        np.linspace(1, 10, n),
        indexing = 'ij'
    )
targets_grid = targets_grid_x * targets_grid_y

In [ ]:
targets = np.vstack(
    (targets_grid.flatten(), np.rot90(targets_grid, k=3).flatten(), np.rot90(targets_grid).flatten()),
)

In [ ]:
print(np.hstack(targets).shape)

Normalize the data to a unit hypercube.

In [ ]:
targets_norm = (targets - np.min(targets)) / (np.max(targets) - np.min(targets))
print(np.min(targets_norm), np.max(targets_norm), targets_norm.shape)

In [ ]:
fig, ax = plt.subplots(1,3,figsize = (14,4))
ax[0].imshow(targets[0,:].reshape(n,n))
ax[0].set_title('$\kappa$', fontsize = 15)
ax[1].imshow(targets[1,:].reshape(n,n))
ax[1].set_title("g1", fontsize = 15)
ax[2].imshow(targets[2,:].reshape(n,n))
ax[2].set_title("g2", fontsize = 15)
plt.show()

Explicitly define the target matrices.
Also add leading unitary dimension to `targets_muygpys` for things to work.

In [ ]:
targets_analytic = targets_norm
targets_muygps = targets_norm[None,:]
print(targets_analytic.shape, targets_muygps.shape)

Having defined the `K*` matrices above, compare posterior means, sans-optimization. 
This comes from (3.4) in Muyskens et al. (2021), which looks like:
$$ \hat{Y}(X^*|X) = K_{\theta}(X^*,X)(K_{\theta}(X,X)+\epsilon)^{-1}Y(X) $$
where $\epsilon = \sigma^2 I$ with $\sigma^2$ being the noise variance.

Analytic: for the analytic implementation, I'll do things with the full "flattened" difference tensors.

In [ ]:
Kcross_analytic_full = original_shear(features, features, length_scale=length_scale)

In [ ]:
print(Kcross_analytic_full.shape, Kin_analytic.shape, targets_analytic.shape)

In [ ]:
def conventional_mean(Kin, Kcross, targets, noise=1e-4):
    return (
        Kcross @ np.linalg.solve(
            Kin + noise * np.identity(3 * n**2),
            targets.reshape(3 * n**2),
        )
    ).reshape(3, n**2)

In [ ]:
posterior_mean_analytic = conventional_mean(
    Kin_analytic,
    Kcross_analytic_full,
    targets_analytic,
)

MuyGPyS implementation using nearest neighbors.

In [ ]:
# create MuyGPS object
shear_model = MuyGPS(
        kernel=ShearKernel(
            deformation=Isotropy(
                F2,
                length_scale=Parameter(length_scale),
            ),
        ),
        noise = HomoscedasticNoise(1e-4)
    )

Create flat solve using MuyGPyS functions.
This should be very close to the analytic solution.

In [ ]:
Kin_flat = Kin_muygps.reshape(3 * n**2, 3 * n**2)
crosswise_diffs_full = _crosswise_differences(features, features)
Kcross_muygps_full = shear_model.kernel(crosswise_diffs_full, adjust=False)
Kcross_flat_full = Kcross_muygps_full.reshape(3 * n**2, 3 * n**2)

In [ ]:
print(
    np.allclose(Kin_analytic, Kin_flat),
    np.allclose(Kcross_analytic_full, Kcross_flat_full),
)

In [ ]:
def show_im(vec, ax):
    im = ax.imshow(vec.reshape(n,n), norm=LogNorm())
    fig.colorbar(im, ax=ax)

def compare_predictions(truth, first, second, fname, sname, fontsize=12):
    f_residual = np.abs(truth - first) + 1e-15
    s_residual = np.abs(truth - second) + 1e-15
    fs_residual = np.abs(first - second) + 1e-15

    fig, ax = plt.subplots(6, 3, figsize = (10, 18))
    
    for axis_set in ax:
        for axis in axis_set:
            axis.set_xticks([])
            axis.set_yticks([])

    ax[0, 0].set_title("$\kappa$")
    ax[0, 1].set_title("g1")
    ax[0, 2].set_title("g2")
    ax[0, 0].set_ylabel("Truth", fontsize=fontsize)
    ax[1, 0].set_ylabel(f"{fname} Mean", fontsize=fontsize)
    ax[2, 0].set_ylabel(f"|truth - {fname}|", fontsize=fontsize)
    ax[3, 0].set_ylabel(f"{sname} Mean", fontsize=fontsize)
    ax[4, 0].set_ylabel(f"|truth - {sname}|", fontsize=fontsize)
    ax[5, 0].set_ylabel(f"|{fname} - {sname}|", fontsize=fontsize)

    # truth
    ax[0, 0].imshow(truth[0,:].reshape(n,n))
    ax[0, 1].imshow(truth[1,:].reshape(n,n))
    ax[0, 2].imshow(truth[2,:].reshape(n,n))

    # first model
    ax[1, 0].imshow(first[0,:].reshape(n,n))
    ax[1, 1].imshow(first[1,:].reshape(n,n))
    ax[1, 2].imshow(first[2,:].reshape(n,n))
    
    # first model residual
    show_im(f_residual[0, :], ax=ax[2, 0])
    show_im(f_residual[1, :], ax=ax[2, 1])
    show_im(f_residual[2, :], ax=ax[2, 2])

    # second model
    ax[3, 0].imshow(second[0,:].reshape(n,n))
    ax[3, 1].imshow(second[1,:].reshape(n,n))
    ax[3, 2].imshow(second[2,:].reshape(n,n))
    
    # second model residual
    show_im(s_residual[0, :], ax=ax[4, 0])
    show_im(s_residual[1, :], ax=ax[4, 1])
    show_im(s_residual[2, :], ax=ax[4, 2])

    # residual between the two models
    show_im(fs_residual[0, :], ax=ax[5, 0])
    show_im(fs_residual[1, :], ax=ax[5, 1])
    show_im(fs_residual[2, :], ax=ax[5, 2])

    plt.show()

In [ ]:
posterior_mean_flat = conventional_mean(
    Kin_flat,
    Kcross_flat_full,
    targets_analytic,
)

In [ ]:
compare_predictions(targets_norm, posterior_mean_flat, posterior_mean_analytic, "flat", "analytic")

In [ ]:
def muygps_mean_workflow(nn_count = 50):
    nn_count = 150
    nbrs_lookup = NN_Wrapper(features, nn_count, nn_method='exact', algorithm='ball_tree')

    features_count = features.shape[0]

    indices = np.arange(features_count)
    test_nn_indices, _ = nbrs_lookup.get_nns(features)

    (
        crosswise_diffs,
        pairwise_diffs,
        nn_targets,
    ) = make_predict_tensors(
        indices,
        test_nn_indices,
        features,
        features,
        targets_norm.swapaxes(0,1),
    )

    nn_targets= nn_targets.swapaxes(-2, -1)

    Kcross = shear_model.kernel(crosswise_diffs)
    K = shear_model.kernel(pairwise_diffs)

    return shear_model.posterior_mean(K, Kcross, nn_targets).swapaxes(0,1)

In [ ]:
posterior_mean_muygps = muygps_mean_workflow(nn_count=50)

Check numerically if things are close.

In [ ]:
np.allclose(posterior_mean_analytic, posterior_mean_muygps)

In [ ]:
compare_predictions(targets_norm, posterior_mean_muygps, posterior_mean_analytic, "MuyGPyS", "Analytic")